<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering   

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.22.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.22.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Intelligence Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather the Information & Intelligence in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks. 

In [3]:
// New prompt to help with decision-making frameworks
var simpleDecisionPrompt = """
Identify and list various decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

Certainly! There are several decision-making frameworks that can enhance the quality of decisions by providing structured approaches to analyze and reason through different scenarios. Here are a few notable ones:

### 1. **SWOT Analysis**
- **Description**: SWOT stands for Strengths, Weaknesses, Opportunities, and Threats. This framework helps in identifying and analyzing the internal and external factors that can impact a decision.
- **Support for Better Analysis**: By categorizing these factors, SWOT helps decision-makers understand the situation thoroughly, recognize potential risks and opportunities, and leverage strengths while mitigating weaknesses.

### 2. **Cost-Benefit Analysis (CBA)**
- **Description**: CBA involves comparing the total expected costs versus the benefits of one or more actions to determine the best or most profitable option.
- **Support for Better Analysis**: This helps ensure that resources are used efficiently and the option with the greatest net benefits is

### Step 3 - Execute a Decision Intelligence Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [5]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Certainly! Here are several decision-making frameworks along with brief descriptions of how each supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT Analysis helps assess internal and external factors that can impact a decision.
   - **Support**: It provides a comprehensive view of a situation by identifying strengths and weaknesses within an organization and opportunities and threats in the external environment. This can help formulate strategic decisions that leverage strengths and opportunities while mitigating weaknesses and threats.

2. **Pareto Analysis (80/20 Rule)**
   - **Description**: Pareto Analysis helps prioritize potential causes of problems to focus efforts where they will have the greatest impact.
   - **Support**: By identifying the 20% of causes that are responsible for 80% of the problems or benefits, decision-makers can effectively allocate resources and efforts to the areas that w

### Step 4 - Execute a Decision Intelligence Prompt with a Custom Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

In [8]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are several decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each framework supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description:** SWOT analysis helps in identifying internal and external factors that are favorable and unfavorable to achieving an objective.
   - **Support:** It provides a structured approach to evaluate the current situation and future potential, helping decision-makers to leverage strengths, address weaknesses, capitalize on opportunities, and mitigate threats.

2. **PEST Analysis (Political, Economic, Social, Technological)**
   - **Description:** PEST analysis examines the external macro-environmental factors that can impact an organization.
   - **Support:** By understanding these external factors, decision-makers can anticipate changes in the environment and adapt their strategies accordingly, ensuring they are proa

### Step 5 - Execute a Decision Instelligence Prompt with a System Prompt (Custom AI Persona)

All the typical best practices of prompt engineering apply when using Semantic Kernel.  
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something wrong
* Provide context (additional information) to the AI
* Using Roles in chat completion prompts
* Give your AI words of encouragement  

One best practice is to provide a common behaviour across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. These prompts can be dynamically created with a programming language like C# or Python.  

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. The decision framework responses are much more robust with decision intelligence information.

In [11]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var simpleDecisionPrompt = "Provide some decision frameworks that can help improve the quality of decisions.";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("Prompt To Send to the Azure OpenAI Chat Completion Service...");
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("Response from the Azure OpenAI Chat Completion Service...");


await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Prompt To Send to the Azure OpenAI Chat Completion Service...
System Prompt: You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Request from the user: Provide some decision frameworks that can help improve the quality of decisions.

----------------------------------------

Response from the Azure OpenAI Chat Completion Service...
Certainly! Decision frameworks are structured approaches that can help improve the quality of decisions by providing a systematic way to evaluate options and outcomes. Here are some widely-used decision frameworks:

### 1. **SWOT Analysis**
**Strengths, Weaknesses, Opportunities, Threats**
- **Purpose:** To evaluate internal and external factors that can impact a decision.
- **Steps:**
  1. Identify internal strengths and weaknesses.
  2. Identify external opportunities and threats.
